# Analisi Performance Consistence Hashing Cassandra

### Installazione Librerie

In [1]:
pip install cassandra-driver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 2.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Connessione al Cluster

In [2]:
import json
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import SimpleStatement

import time
# Connessione al cluster Cassandra
auth_provider = PlainTextAuthProvider(username='admin', password='admin')
cluster = Cluster(['cassandra-container1'], port=9042, auth_provider=auth_provider)
session = cluster.connect()
print("Connessione effettuata correttamente")

Connessione effettuata correttamente


### Creazione KeySpace e Tabelle

In [3]:
# Creazione del keyspace se non esiste
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS comuni_keyspace
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '2'}
""")

# Connessione al keyspace
session.set_keyspace('comuni_keyspace')
print("Keyspace creato correttamente")


Keyspace creato correttamente


In [4]:
# Creazione della tabella per memorizzare i dati JSON
session.execute("""
    CREATE TABLE IF NOT EXISTS comuni (
        codice TEXT PRIMARY KEY,
        nome TEXT,
        zona_codice TEXT,
        zona_nome TEXT,
        regione_codice TEXT,
        regione_nome TEXT,
        provincia_codice TEXT,
        provincia_nome TEXT,
        sigla TEXT,
        codiceCatastale TEXT,
        cap LIST<TEXT>,
        popolazione INT
    )
""")



### Test Scrittura e Lettura

In [5]:
#Scrittura di un dato
# Inserimento di un singolo record nella tabella comuni
session.execute(
    """
    INSERT INTO comuni (codice, nome, zona_codice, zona_nome, regione_codice, regione_nome,
                        provincia_codice, provincia_nome, sigla, codiceCatastale, cap, popolazione)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """,
    (
        "999999",  # codice
        "ComuneTest",  # nome
        "2",  # zona_codice
        "Nord-est",  # zona_nome
        "05",  # regione_codice
        "Veneto",  # regione_nome
        "028",  # provincia_codice
        "Padova",  # provincia_nome
        "PD",  # sigla
        "A002",  # codiceCatastale
        ["99999"],  # cap
        11370  # popolazione
    )
)

print("Record inserito correttamente.")


Record inserito correttamente.


In [6]:
#lettura di un dato
codice_da_leggere = "028001"  # Codice del comune da leggere

row = session.execute(
    "SELECT * FROM comuni WHERE codice=%s", (codice_da_leggere,)
).one()

if row:
    print(f"Comune trovato: {row.nome}, Provincia: {row.provincia_nome}, Popolazione: {row.popolazione}")
else:
    print("Comune non trovato.")


Comune non trovato.


### Scrittura e Lettura massiva

In [7]:
#Scrittura massiva
# Lettura del file JSON
with open('comuni.json', 'r', encoding='utf-8') as file:
    comuni_data = json.load(file)
start_time = time.time()
# Inserimento dei dati nella tabella
for comune in comuni_data:
    session.execute(
        """
        INSERT INTO comuni (codice, nome, zona_codice, zona_nome, regione_codice, regione_nome,
                            provincia_codice, provincia_nome, sigla, codiceCatastale, cap, popolazione)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (
            comune['codice'],
            comune['nome'],
            comune['zona']['codice'],
            comune['zona']['nome'],
            comune['regione']['codice'],
            comune['regione']['nome'],
            comune['provincia']['codice'],
            comune['provincia']['nome'],
            comune['sigla'],
            comune['codiceCatastale'],
            comune['cap'],
            comune['popolazione']
        )
    )
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Dati inseriti correttamente nel database Cassandra in {elapsed_time:.2f} secondi.")

Dati inseriti correttamente nel database Cassandra in 24.73 secondi.


In [ ]:
# Lettura massiva di tutti i record dalla tabella comuni
start_time = time.time()
rows = session.execute("SELECT * FROM comuni")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Dati Letti correttamente dal database Cassandra in {elapsed_time:.2f} secondi.")
print("Lettura di tutti i comuni:")
for row in rows:
    print(f"Codice: {row.codice}, Nome: {row.nome}, Provincia: {row.provincia_nome}, Popolazione: {row.popolazione}")

### Altre operazioni

In [8]:
#Altre operazioni di test
# Aggiornamento del nome e della popolazione di un comune specifico
codice_da_aggiornare = "028001"  # Codice del comune da aggiornare
nuovo_nome = "Nuovo Nome Comune"
nuova_popolazione = 5000

start_time = time.time()

session.execute(
    """
    UPDATE comuni
    SET nome = %s, popolazione = %s
    WHERE codice = %s
    """, 
    (nuovo_nome, nuova_popolazione, codice_da_aggiornare)
)

end_time = time.time()
execution_time = end_time - start_time

print(f"Comune con codice {codice_da_aggiornare} aggiornato con successo.")
print(f"Tempo di esecuzione: {execution_time:.6f} secondi")


Comune con codice 028001 aggiornato con successo.
Tempo di esecuzione: 0.005967 secondi


In [9]:
#Rimozione dato
# Cancellazione di un comune dalla tabella
codice_da_cancellare = "028001"  # Codice del comune da cancellare

start_time = time.time()

session.execute(
    "DELETE FROM comuni WHERE codice=%s", (codice_da_cancellare,)
)

end_time = time.time()
execution_time = end_time - start_time

print(f"Comune con codice {codice_da_cancellare} cancellato con successo.")
print(f"Tempo di esecuzione: {execution_time:.6f} secondi")


Comune con codice 028001 cancellato con successo.
Tempo di esecuzione: 0.003596 secondi


In [10]:

# Lettura di tutti i comuni di una specifica regione
regione_da_leggere = "Lombardia"

start_time = time.time()

rows = session.execute(
    "SELECT * FROM comuni WHERE regione_nome=%s ALLOW FILTERING", (regione_da_leggere,)
)

end_time = time.time()
execution_time = end_time - start_time

print(f"Comuni nella regione {regione_da_leggere}:")
for row in rows:
    print(f"{row.nome}, Provincia: {row.provincia_nome}, Popolazione: {row.popolazione}")

print(f"Tempo di esecuzione: {execution_time:.6f} secondi")


Comuni nella regione Lombardia:
Oltre il Colle, Provincia: Bergamo, Popolazione: 1058
Trezzone, Provincia: Como, Popolazione: 237
Cigole, Provincia: Brescia, Popolazione: 1619
Oltrona di San Mamette, Provincia: Como, Popolazione: 2289
Castana, Provincia: Pavia, Popolazione: 744
Santa Giuletta, Provincia: Pavia, Popolazione: 1685
Schignano, Provincia: Como, Popolazione: 874
Montecalvo Versiggia, Provincia: Pavia, Popolazione: 561
Olmeneta, Provincia: Cremona, Popolazione: 967
Madesimo, Provincia: Sondrio, Popolazione: 540
Bene Lario, Provincia: Como, Popolazione: 342
Sulbiate, Provincia: Monza e della Brianza, Popolazione: 4067
Cuasso al Monte, Provincia: Varese, Popolazione: 3604
Massalengo, Provincia: Lodi, Popolazione: 4192
Tignale, Provincia: Brescia, Popolazione: 1298
Casteggio, Provincia: Pavia, Popolazione: 6836
Lallio, Provincia: Bergamo, Popolazione: 4124
Volpara, Provincia: Pavia, Popolazione: 133
Griante, Provincia: Como, Popolazione: 623
Sabbio Chiese, Provincia: Brescia, Po

In [12]:
#funzione per eliminare il keyspace
def elimina_keyspace(keyspace_da_eliminare):
    # Inizio del calcolo del tempo
    start_time = time.time()
    
    # Comando per eliminare il keyspace
    session.execute(f"DROP KEYSPACE IF EXISTS {keyspace_da_eliminare}")
    
    # Fine del calcolo del tempo
    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"Keyspace '{keyspace_da_eliminare}' è stato eliminato.")
    print(f"Tempo di esecuzione: {execution_time:.6f} secondi")

#eliminazione keyspace 
elimina_keyspace('comuni_keyspace')


Keyspace 'comuni_keyspace' è stato eliminato.
Tempo di esecuzione: 3.174936 secondi
